# Clustering To Explore Neighbourhoods
## Review
In the last post, we reviewed some of the visualizations we made via datashader and learned a few things about the neighbourhoods in NYC. As someone who is not from NYC, has never lived in NYC, and has only ever visited Manhattan and Brooklyn (and even then, my exposure is quite limited), I was really amazed at how hard of a boundary there were between different neighbourhoods and different types / severity of offenses! I don't think the takeaways are mindblowing as I'm sure someone living in the area is familiar with the general safety of the boroughs, but I came to these conclusions with a MacBook, some chips and guac, and a couch in the comfort of my home here in Edmonton, Alberta... the NY of Canada.

![](https://a1.cdn-hotels.com/cos/production183/d888/1c4131d0-86eb-11e6-a741-0242ac110051.jpg)

The resemblance is _**uncanny**_.

Point being I'm learning via data, and data alone. I'm not saying that looking at this data is a substitute for being there either - By not being there, I have to be comfortable with the fact that I have no supporting information _**outside**_ of the data. This dataset only has a handful of columns, but surely there is information which is not feasible for the NYPD to collect. In fact, there's probably data that the NYPD has not even made open to the public! If we think about something like Grand Larceny, there can be so many other factors that may describe the crime and tell a story within itself... the square footage of the unit, the value of goods stolen, the ethnicity / race of the criminal / victim... there are pretty much endless data points we can capture if we want to get really technical and creative, but obviously there is a balance between how much data you _**can**_ collect vs how much data you _**do**_ collect.

Regardless, it's pretty cool to be learning through data, and data alone. Reminds me of when I tried to [predict the All-NBA players](https://strikingmoose.com/2017/07/31/predicting-2016-2017-all-nba-teams-end-to-end/). If I knew how to run a gradient boosted tree, I basically didn't have to know _**anything**_ about basketball. A scary though, and obviously not always the right train of thought, but always crazy to see where data, alone, can pave the way.

## Contextualizing Visualizations
I want to take a few minutes to write about the use of visualiations, and why the title of this post is on clustering.

The datashader visualizations were beautiful and really informative, but visualizations themselves have pitfalls as well - We gain simplicity and clarity by eliminating detail. I mean, isn't that the whole point of a visualization anyways? Why use a visualization if the data was simple? We make visualizations in the first place because our dataset is too complicated to look at in a tabular form! The inner workings of datashader is based on this principal. Instead of looking at each point individually, datashader aggregates by a more feasible approach for a computer screen... the pixel! Each pixel is summarized based on the data points that fall into that pixel's boundary, and the data is summarized that way.

In datashader, we use the **eq\_hist** color method which tries to let each shade of color in our color gradient / map represent an equal _**amount of samples**_ in our entire dataset. When we apply this coloring method, we lose a realistic handle on the magnitude of our data. eq\_hist is literally _**manipulating the values of the data to appease our eyes**_.

When we categorize the data by the offense level or offense type, we are only looking at 3-4 categories at a time. What about all the other categories? For now, we've just left them out of the visualization altogether. Is this the right thing to do? Well, it really depends on the question we're trying to answer, doesn't it? I realize I came into this entire exercise without any real objectives other than to just learn a bit more about crime in NYC, but when I was using datashader, I was inherently doing a geographical analysis. The map we generated below

<img src="https://s3.ca-central-1.amazonaws.com/2017edmfasatb/nypd_complaints/images/35_nypd_complaints_offense_type_edited.png" width="500">

showed 3 / 10 possible values we had for the OFFENSE\_DESCRIPTION column (which, itself, was grouped down from 50+ categories). By ignoring the data, we are making the same conscious decision that we made when we decided to visualize in the first place. We want to look at the data through a _**different lens**_ in order to get a different perspective. Whether that perspective is the right one to take to answer that specific question... that's where the art of it comes into play. When I created that graph, I had a general sense that wealthier neighbourhoods experienced more Grand Larceny and lesser well-off neighbourhoods experienced more Dangerous Drug offenses while Harrassment was widespread throughout all of NYC. That visualization worked out perfectly for me because the boundaries were so clearly defined across the 3 offense types. The reality of the situation is that I feel uneasy plotting anything over 3-4 categories per map. Can you imagine a map with 10 categories, let alone 50? Can we even process all the colors? Can we even fit a legend on the plot? Visualization inherently has the ability to only summarize a limited amount of information due to our own limitations as humans.

## Clustering
I'm now feeling a bit uneasy because of I've ignored so many types of offenses... But wait, didn't I ignore them in the first place so I could make my visualization simpler to understand? Man... this is confusing...

<img src="https://i.giphy.com/media/l2R01mSIsazqNQ7ks/giphy.webp" width="400">

You know, it just comes down the fact that I'm not coming into this project with any specific objective. I'm looking for some type of finding, but I don't know which 8 questions I need to ask to get there. I'm digging for gold, but I haven't a clue where to look or what tools I need... hell... I don't even know if it's _**gold**_ I'm looking for! That's what this blog is for right...? To have a sandbox where nobody judges me and I can fail as much as I want!

<img src="https://media.collegetimes.com/uploads/2016/02/09152603/laughing-then-crying.gif" width="500">

Because I'm feeling so sensitive to those offenses that I ignored, I want to find a middle ground between looking at the tabular data and and looking at a limited number of categories on a map.

In comes clustering.

When I was checking out one of the [Kaggle NYC Taxi competition kernels](https://www.kaggle.com/gaborfodor/from-eda-to-the-top-lb-0-367), I noticed a super interesting use of clustering. Yes, yes, blah, blah I'm naive, I get it, but it still was really cool for me to see. About a third of a way down the notebook, the kaggler uses K-means clustering to actually cluster all the taxi data into _**regions**_, and these regions loosely represent a small neighbourhood in NYC. Ideally, we would have the boundaries of each neighbourhood defined in code so we can explicitly define each neighbourhood rather than _**approximating**_ their definition, but in the case of taxis, the hard boundaries may not matter anyways because taxis run in and out of neighbourhoods continuously. Oh yea, also we don't have the boundaries in code, so... yeah. This kaggler uses clustering to see if the neighbourhood the taxis are leaving from has a huge impact on the trip duration.

In my case, I could also use clustering to define my data into 20 or so clusters. If I approximate NYC as 20 clusters of neighbourhoods, I might be able to better _**view**_ the data. Again, we're losing detail here as we're summarizing 5 million points into 20 clusters, but it will give us more detail than the map did as we'll be able to just look at the offense statistics / averages for a manageable amount of data samples (one row of data for each cluster).

## Spark ML
Because we are working with the full dataset again, we need to (or want to because our dataset isn't that big anyways) bring Spark back. Another reason for Spark's dominance is that it comes with a _**distributed, out of the box, easy to use machine learning library, Spark ML**_. Spark ML has a K-means clustering algorithm built in - lucky for us!